In [ ]:
from kubernetes import client


from kfserving import constants
from kfserving import KFServingClient
from kfserving import V1beta1PredictorSpec
from kfserving import V1beta1InferenceService
from kfserving import V1beta1InferenceServiceSpec
from kfserving import V1beta1TFServingSpec
from kfserving import utils


from kubernetes.client import V1ResourceRequirements

import os
import sys
import argparse
import logging
import time



## kfserving module을 이용해 python code로 직접 클러스터에 serving을 하는 과정
class KFServing(object):
    
    
    def run(self):
        parser = argparse.ArgumentParser()
        parser.add_argument('--namespace', required=False, default='kubeflow')
        parser.add_argument('--storage_uri', required=False, default='/mnt/export')
        parser.add_argument('--name', required=False, default='kfserving-sample')        
        args = parser.parse_args()
        namespace = args.namespace
        serving_name =  args.name
        api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_V1BETA1
                                      
        
        ## inference server spec 정의
        predictor=V1beta1PredictorSpec(tensorflow=V1beta1TFServingSpec(
                                      storage_uri=args.storage_uri,
                                      resources=V1ResourceRequirements(
                                          requests={'cpu':'100m','memory':'1Gi'},
                                          limits={'cpu':'100m', 'memory':'1Gi'})))
        
        
        ## kfserving crd(inferenceservice) 정의       
        
        isvc = V1beta1InferenceService(api_version=api_version,
                                   kind=constants.KFSERVING_KIND,
                                   metadata=client.V1ObjectMeta(
                                       name=serving_name, namespace=namespace),
                                   spec=V1beta1InferenceServiceSpec(predictor=predictor))
        
        KFServing = KFServingClient()
        
        
        KFServing.create(isvc)
        print('waiting 5 sec for Creating InferenceService')
        time.sleep(5)
        
        KFServing.get(serving_name, namespace=namespace, watch=True, timeout_seconds=300)

## serving하는 python 코드를 fairing으로 imaging 하는 작업
if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:      
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils        
        from kubeflow.fairing.builders.cluster.cluster import ClusterBuilder
        from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource
        from kubeflow.fairing.preprocessors.converted_notebook import ConvertNotebookPreprocessor
        
        ## 이미지를 배포하기 위한 auth정보 등록
        ! kubectl delete cm docker-config
        ! kubectl create cm docker-config --from-file=/home/jovyan/.docker/config.json

        DOCKER_REGISTRY = 'kitaeyoo777'
        base_image = 'kitaeyoo777/kubeflow-kfserving:test6'
        image_name = 'kfserving'

        minio_context_source = MinioContextSource(endpoint_url='http://minio-service.kubeflow.svc.cluster.local:9000', 
                                                  minio_secret='minio', 
                                                  minio_secret_key='minio123', 
                                                  region_name='us-east-1')
        builder = ClusterBuilder(
            registry=DOCKER_REGISTRY,
            image_name=image_name,
            base_image=base_image,
            context_source=minio_context_source,
            push=True,
            preprocessor=ConvertNotebookPreprocessor(
                notebook_file="KFServing-fairing.ipynb"
            )
        )
        builder.build()       
        
        
    else:
        serving = KFServing()
        serving.run()

configmap "docker-config" deleted
configmap/docker-config created


[I 211118 06:15:45 cluster:46] Building image using cluster builder.
[I 211118 06:15:45 base:107] Creating docker context: /tmp/fairing_context_dog63kd2
[I 211118 06:15:45 converted_notebook:127] Converting KFServing-fairing.ipynb to KFServing-fairing.py
[W 211118 06:15:45 manager:298] Waiting for fairing-builder-sp6rt-srsj6 to start...
[W 211118 06:15:46 manager:298] Waiting for fairing-builder-sp6rt-srsj6 to start...
[W 211118 06:15:46 manager:298] Waiting for fairing-builder-sp6rt-srsj6 to start...
[W 211118 06:15:46 manager:298] Waiting for fairing-builder-sp6rt-srsj6 to start...
[I 211118 06:15:46 manager:304] Pod started running True
[I 211118 06:15:47 manager:304] Pod started running True


INFO[0004] Retrieving image manifest kitaeyoo777/kubeflow-kfserving:test6
INFO[0006] Retrieving image manifest kitaeyoo777/kubeflow-kfserving:test6
INFO[0008] Built cross stage deps: map[]
INFO[0008] Retrieving image manifest kitaeyoo777/kubeflow-kfserving:test6
INFO[0009] Retrieving image manifest kitaeyoo777/kubeflow-kfserving:test6
INFO[0011] Executing 0 build triggers
INFO[0011] Unpacking rootfs as cmd RUN if [ -e requirements.txt ];then pip install --no-cache -r requirements.txt; fi requires it.
